# **Problem Statement**




To find the hidden Assocation between items purchased in online store company and to fid out the rules among the each items. To Provide the business these rules to make decision about the items placement s and to recommend the items more effective way for cross sell and upsell.





In [ ]:
#Importing packages
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity ="all"

# Importing the data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link = 'https://drive.google.com/open?id=1R3S7oxkb1aLwOMf7UCb1tcdO8HiMsL9w'

In [ ]:
fluff, id = link.split('=')
print (id)

1R3S7oxkb1aLwOMf7UCb1tcdO8HiMsL9w


In [ ]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Online _Store.csv')  
df3 = pd.read_csv('Online _Store.csv',header=0,encoding = 'unicode_escape')

In [ ]:
df3.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


# Finding the countries which are available in data sets and picked country 'Spain' for Analysis

In [ ]:
df3.Country.nunique()
df3.Country.unique()

38

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [ ]:
data = df3[df3.Country=='Spain']
data.shape

(2533, 8)

In [ ]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
6421,536944,22383,LUNCH BAG SUKI DESIGN,70,12/3/2010 12:20,1.65,12557.0,Spain
6422,536944,22384,LUNCH BAG PINK POLKADOT,100,12/3/2010 12:20,1.45,12557.0,Spain
6423,536944,20727,LUNCH BAG BLACK SKULL.,60,12/3/2010 12:20,1.65,12557.0,Spain
6424,536944,20725,LUNCH BAG RED RETROSPOT,70,12/3/2010 12:20,1.65,12557.0,Spain
6425,536944,20728,LUNCH BAG CARS BLUE,100,12/3/2010 12:20,1.45,12557.0,Spain


# Creating Pivot Table

Creating a Pivot table by making index as invoice no and columns as Description and Quantity as values for getting each customer purchases in different columns

In [ ]:
basket = pd.pivot_table(data,index = 'InvoiceNo',columns ='Description',values ='Quantity',fill_value = 0)

In [ ]:
basket.head(10)

Description,50'S CHRISTMAS GIFT BAG LARGE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,10 COLOUR SPACEBOY PEN,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,20 DOLLY PEGS RETROSPOT,200 RED + WHITE BENDY STRAWS,3 DRAWER ANTIQUE WHITE WOOD CABINET,3 HOOK HANGER MAGIC GARDEN,3 HOOK PHOTO SHELF ANTIQUE WHITE,3 PIECE SPACEBOY COOKIE CUTTER SET,3 RAFFIA RIBBONS 50'S CHRISTMAS,3 ROSE MORRIS BOXED CANDLES,3 STRIPEY MICE FELTCRAFT,3 TIER CAKE TIN GREEN AND CREAM,3 TIER CAKE TIN RED AND CREAM,3 TRADITIONAl BISCUIT CUTTERS SET,3 WHITE CHOC MORRIS BOXED CANDLES,36 DOILIES DOLLY GIRL,36 DOILIES VINTAGE CHRISTMAS,36 FOIL HEART CAKE CASES,36 FOIL STAR CAKE CASES,36 PENCILS TUBE RED RETROSPOT,36 PENCILS TUBE SKULLS,36 PENCILS TUBE WOODLAND,3D CHRISTMAS STAMPS STICKERS,3D TRADITIONAL CHRISTMAS STICKERS,3D VINTAGE CHRISTMAS STICKERS,4 TRADITIONAL SPINNING TOPS,5 HOOK HANGER RED MAGIC TOADSTOOL,6 EGG HOUSE PAINTED WOOD,6 GIFT TAGS 50'S CHRISTMAS,6 GIFT TAGS VINTAGE CHRISTMAS,6 RIBBONS ELEGANT CHRISTMAS,6 RIBBONS EMPIRE,6 RIBBONS RUSTIC CHARM,6 RIBBONS SHIMMERING PINKS,...,WOOD STAMP SET THANK YOU,WOODEN ADVENT CALENDAR RED,WOODEN BOX OF DOMINOES,WOODEN CROQUET GARDEN SET,WOODEN FRAME ANTIQUE WHITE,WOODEN HAPPY BIRTHDAY GARLAND,WOODEN HEART CHRISTMAS SCANDINAVIAN,WOODEN PICTURE FRAME WHITE FINISH,WOODEN ROUNDERS GARDEN SET,WOODEN SCHOOL COLOURING SET,WOODEN STAR CHRISTMAS SCANDINAVIAN,WOODEN TREE CHRISTMAS SCANDINAVIAN,WOODEN UNION JACK BUNTING,WOODLAND HEIGHT CHART STICKERS,WOODLAND CHARLOTTE BAG,WOODLAND DESIGN COTTON TOTE BAG,WOODLAND MINI BACKPACK,WOODLAND SMALL BLUE FELT HEART,WOODLAND SMALL RED FELT HEART,WORLD WAR 2 GLIDERS ASSTD DESIGNS,WRAP VINTAGE DOILY,WRAP ALPHABET DESIGN,WRAP CIRCUS PARADE,WRAP DOILEY DESIGN,WRAP DOLLY GIRL,WRAP ENGLISH ROSE,WRAP FLOWER SHOP,WRAP MAGIC FOREST,WRAP MONSTER FUN,WRAP PAISLEY PARK,WRAP RED APPLES,WRAP RED VINTAGE DOILY,WRAP SUKI AND FRIENDS,"WRAP, BILLBOARD FONTS DESIGN",YELLOW COAT RACK PARIS FASHION,YELLOW FLOWERS FELT HANDBAG KIT,ZINC FOLKART SLEIGH BELLS,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
536944,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
538095,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
538525,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
539351,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
540469,0,0,8,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,25,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0
540550,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,0,...,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
540785,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
540801,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,0,...,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
541491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,12,0,0,0,0,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Making all purchsed items as '1' and Non Puchasesd items as '0' for ecah customers

In [ ]:
#Write a function to do 0 or 1 for all items
def cod(x):
    if x <= 0:
        return 0
    if x >= 0:
        return 1
basket_Sets = basket.applymap(cod)

In [ ]:
basket_Sets.head()

Description,50'S CHRISTMAS GIFT BAG LARGE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,10 COLOUR SPACEBOY PEN,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,20 DOLLY PEGS RETROSPOT,200 RED + WHITE BENDY STRAWS,3 DRAWER ANTIQUE WHITE WOOD CABINET,3 HOOK HANGER MAGIC GARDEN,3 HOOK PHOTO SHELF ANTIQUE WHITE,3 PIECE SPACEBOY COOKIE CUTTER SET,3 RAFFIA RIBBONS 50'S CHRISTMAS,3 ROSE MORRIS BOXED CANDLES,3 STRIPEY MICE FELTCRAFT,3 TIER CAKE TIN GREEN AND CREAM,3 TIER CAKE TIN RED AND CREAM,3 TRADITIONAl BISCUIT CUTTERS SET,3 WHITE CHOC MORRIS BOXED CANDLES,36 DOILIES DOLLY GIRL,36 DOILIES VINTAGE CHRISTMAS,36 FOIL HEART CAKE CASES,36 FOIL STAR CAKE CASES,36 PENCILS TUBE RED RETROSPOT,36 PENCILS TUBE SKULLS,36 PENCILS TUBE WOODLAND,3D CHRISTMAS STAMPS STICKERS,3D TRADITIONAL CHRISTMAS STICKERS,3D VINTAGE CHRISTMAS STICKERS,4 TRADITIONAL SPINNING TOPS,5 HOOK HANGER RED MAGIC TOADSTOOL,6 EGG HOUSE PAINTED WOOD,6 GIFT TAGS 50'S CHRISTMAS,6 GIFT TAGS VINTAGE CHRISTMAS,6 RIBBONS ELEGANT CHRISTMAS,6 RIBBONS EMPIRE,6 RIBBONS RUSTIC CHARM,6 RIBBONS SHIMMERING PINKS,...,WOOD STAMP SET THANK YOU,WOODEN ADVENT CALENDAR RED,WOODEN BOX OF DOMINOES,WOODEN CROQUET GARDEN SET,WOODEN FRAME ANTIQUE WHITE,WOODEN HAPPY BIRTHDAY GARLAND,WOODEN HEART CHRISTMAS SCANDINAVIAN,WOODEN PICTURE FRAME WHITE FINISH,WOODEN ROUNDERS GARDEN SET,WOODEN SCHOOL COLOURING SET,WOODEN STAR CHRISTMAS SCANDINAVIAN,WOODEN TREE CHRISTMAS SCANDINAVIAN,WOODEN UNION JACK BUNTING,WOODLAND HEIGHT CHART STICKERS,WOODLAND CHARLOTTE BAG,WOODLAND DESIGN COTTON TOTE BAG,WOODLAND MINI BACKPACK,WOODLAND SMALL BLUE FELT HEART,WOODLAND SMALL RED FELT HEART,WORLD WAR 2 GLIDERS ASSTD DESIGNS,WRAP VINTAGE DOILY,WRAP ALPHABET DESIGN,WRAP CIRCUS PARADE,WRAP DOILEY DESIGN,WRAP DOLLY GIRL,WRAP ENGLISH ROSE,WRAP FLOWER SHOP,WRAP MAGIC FOREST,WRAP MONSTER FUN,WRAP PAISLEY PARK,WRAP RED APPLES,WRAP RED VINTAGE DOILY,WRAP SUKI AND FRIENDS,"WRAP, BILLBOARD FONTS DESIGN",YELLOW COAT RACK PARIS FASHION,YELLOW FLOWERS FELT HANDBAG KIT,ZINC FOLKART SLEIGH BELLS,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
536944,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
538095,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
538525,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
539351,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
540469,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


# Fitiing Apriori Algorithm

In [ ]:
freq_items = apriori(basket_Sets,min_support = 0.05,use_colnames = True)
freq_items.head()

,support,itemsets
0,0.057143,(4 TRADITIONAL SPINNING TOPS)
1,0.142857,(6 RIBBONS RUSTIC CHARM)
2,0.066667,(ALARM CLOCK BAKELIKE GREEN)
3,0.057143,(ALARM CLOCK BAKELIKE PINK)
4,0.057143,(ALARM CLOCK BAKELIKE RED )


# Support, Confidence and Lift

**Support** :- It helps as to focus on high volume product and product combination whic are prchasesd

Support (A) = Count of Product A/Total No of Transactionn

**Confidence** :- It is defined as conditional Probability of two purchases

Confidence (Antecedent,cosequents) = Support(consequents,Antecedent)/support(Anticedents)

**Lift**

Lift is defined as ratio of the obeserved to the expected support

Lift (A,B) = Support/Support(A)*Support(B)

In [ ]:
#Find the rules with lift more than 1
rules = association_rules(freq_items,metric="lift",min_threshold=1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(4 TRADITIONAL SPINNING TOPS),(POSTAGE),0.057143,0.590476,0.057143,1.000000,1.693548,0.023401,inf
1,(POSTAGE),(4 TRADITIONAL SPINNING TOPS),0.590476,0.057143,0.057143,0.096774,1.693548,0.023401,1.043878
2,(6 RIBBONS RUSTIC CHARM),(ASSORTED COLOUR BIRD ORNAMENT),0.142857,0.114286,0.085714,0.600000,5.250000,0.069388,2.214286
3,(ASSORTED COLOUR BIRD ORNAMENT),(6 RIBBONS RUSTIC CHARM),0.114286,0.142857,0.085714,0.750000,5.250000,0.069388,3.428571
4,(JAM MAKING SET WITH JARS),(6 RIBBONS RUSTIC CHARM),0.133333,0.142857,0.057143,0.428571,3.000000,0.038095,1.500000


In [ ]:
#Finding best support items
rules.sort_values('support',ascending = False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
55,(POSTAGE),(REGENCY CAKESTAND 3 TIER),0.590476,0.209524,0.161905,0.274194,1.308651,0.038186,1.089101
54,(REGENCY CAKESTAND 3 TIER),(POSTAGE),0.209524,0.590476,0.161905,0.772727,1.308651,0.038186,1.801905
10,(6 RIBBONS RUSTIC CHARM),(POSTAGE),0.142857,0.590476,0.123810,0.866667,1.467742,0.039456,3.071429
11,(POSTAGE),(6 RIBBONS RUSTIC CHARM),0.590476,0.142857,0.123810,0.209677,1.467742,0.039456,1.084548
20,(ASSORTED COLOUR BIRD ORNAMENT),(POSTAGE),0.114286,0.590476,0.104762,0.916667,1.552419,0.037279,4.914286
21,(POSTAGE),(ASSORTED COLOUR BIRD ORNAMENT),0.590476,0.114286,0.104762,0.177419,1.552419,0.037279,1.076751
38,(PACK OF 72 RETROSPOT CAKE CASES),(POSTAGE),0.104762,0.590476,0.104762,1.000000,1.693548,0.042902,inf
39,(POSTAGE),(PACK OF 72 RETROSPOT CAKE CASES),0.590476,0.104762,0.104762,0.177419,1.693548,0.042902,1.088329
78,"(6 RIBBONS RUSTIC CHARM, ASSORTED COLOUR BIRD ...",(POSTAGE),0.085714,0.590476,0.085714,1.000000,1.693548,0.035102,inf
79,"(6 RIBBONS RUSTIC CHARM, POSTAGE)",(ASSORTED COLOUR BIRD ORNAMENT),0.123810,0.114286,0.085714,0.692308,6.057692,0.071565,2.878571


# Highest Support Items

POSTAGE and CAKESTAND  are the highest selling items

Postage and Ribbon and cackestand are the higest selling items as a result company need to keep all these items in stock


In [ ]:
rules[ (rules['lift'] >=4)  & (rules['support'] >= 0.05) & (rules['confidence'] >= 0.7)].head(10)



,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3,(ASSORTED COLOUR BIRD ORNAMENT),(6 RIBBONS RUSTIC CHARM),0.114286,0.142857,0.085714,0.750000,5.250000,0.069388,3.428571
13,(RED RETROSPOT TAPE),(6 RIBBONS RUSTIC CHARM),0.066667,0.142857,0.057143,0.857143,6.000000,0.047619,6.000000
30,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER ),0.057143,0.104762,0.057143,1.000000,9.545455,0.051156,inf
35,(SET/5 RED RETROSPOT LID GLASS BOWLS),(JAM MAKING SET WITH JARS),0.057143,0.133333,0.057143,1.000000,7.500000,0.049524,inf
36,(LUNCH BAG RED RETROSPOT),(LUNCH BAG PINK POLKADOT),0.076190,0.095238,0.057143,0.750000,7.875000,0.049887,3.619048
42,(PLASTERS IN TIN CIRCUS PARADE ),(PLASTERS IN TIN SKULLS),0.066667,0.095238,0.057143,0.857143,9.000000,0.050794,6.333333
48,(POPPY'S PLAYHOUSE BEDROOM ),(POPPY'S PLAYHOUSE KITCHEN),0.076190,0.066667,0.066667,0.875000,13.125000,0.061587,7.466667
49,(POPPY'S PLAYHOUSE KITCHEN),(POPPY'S PLAYHOUSE BEDROOM ),0.066667,0.076190,0.066667,1.000000,13.125000,0.061587,inf
66,(RED RETROSPOT CAKE STAND),(SPACEBOY LUNCH BOX ),0.076190,0.114286,0.057143,0.750000,6.562500,0.048435,3.542857
70,(SET/10 PINK POLKADOT PARTY CANDLES),(SET OF 6 GIRLS CELEBRATION CANDLES),0.066667,0.066667,0.057143,0.857143,12.857143,0.052698,6.533333


# Rules  and Recommendations

**Rules :**

*   If assorted color bird ornament is purchased and 75 % of times customer will buy 6 Ribbon rustic charms
*   If Red retro spot tape is purchased Ribbon rustic charm will also purchased in 85 % of time
* If Green tea cup and saucer is purchased then rose color will also purchase all the time.
*if Lid Glass Bowls purchases then Jam Making Jars also purchased in 75 % of the time
*if plasters in circus parade purchased and plasters in tins skulls is purchased in 85 % of the time.
*if playhouse bedroom purchased then play house kitchen also purchased 100 % of the time.
*if cake stand is purchased then lunch box also purchased
*if pink polkadot candles purchased then girls celebration candles is also purchased.


**Recommendations to Marketing Team**

From these rules business can give recommendations to the customers

*   Green tea cup and saucer and rose color cup is purchased so marketing team give online recommendations if customer purchased these green tea cup also purchase red cup and cross sell the product
*   Play hose bedroom can cross sell with play house kitchen in a discount rate.
*Red retrospot tape and Ribbon rustic charm can be cross sell if business will give any offer
*cake stand and lunch box also can be cross sell
*Polladot Candles and Celebration can be bundled together and can be sell in a discount will give more sales.

